In [228]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

In [229]:
data_sample = pd.read_csv("gender_submission.csv")
data_sample.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [230]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
data = pd.concat([train, test])

data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [231]:
data.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


In [232]:
print("Train size:", train.shape)
print("Test size:", test.shape)
print("Data size:", data.shape)

Train size: (891, 12)
Test size: (418, 11)
Data size: (1309, 12)


In [233]:
trainX = train[test.columns]
trainY = train['Survived']
test = test

In [234]:
trainX.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [235]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


### Embarked fill nan

In [236]:
data['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [237]:
data['Embarked'].fillna('S', inplace = True)

### Title + age fill nan

In [238]:
data['Title'] = data['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
data

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Mrs
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Miss
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Mrs
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Mr
5,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877,Mr
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463,Mr
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909,Master
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742,Mrs
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736,Mrs


In [239]:
data['Title'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Ms                2
Major             2
Mlle              2
the Countess      1
Don               1
Capt              1
Lady              1
Jonkheer          1
Mme               1
Dona              1
Sir               1
Name: Title, dtype: int64

In [240]:
data[data['Title']=='Dr'].Age

245    44.0
317    54.0
398    23.0
632    32.0
660    50.0
766     NaN
796    49.0
293    53.0
Name: Age, dtype: float64

In [241]:
Title_counts_df = pd.DataFrame([data['Title'].value_counts()]).T.reset_index()

In [242]:
for elt in Title_counts_df['index']:
    data.loc[data['Title']==elt,'Age'] = data.loc[data['Title']==elt]['Age'].fillna(data.loc[data['Title']==elt].Age.median())

In [243]:
data[:train.shape[0]].Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [244]:
data.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona'], dtype=object)

##### Видим, что в тесте есть Title которых нет в train data set

In [245]:
def get_titles(df):
    
    # we extract the title from each name
    df['Title'] = df['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    
    # a map of more aggregated titles
    Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",                
                        "Lady" :      "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "Mme":        "Mrs",
                        "Ms":         "Mrs",                
                        "Mrs" :       "Mrs",
                        "Mlle":       "Miss",
                        "Miss" :      "Miss",
                        "Mr" :        "Mr",
                        "Master" :    "Master"
                       }
    # we map each title
    df['Title'] = df.Title.map(Title_Dictionary)

In [246]:
get_titles(data)

### Cabin

In [247]:
data['Cabin'].head()

0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object

In [248]:
data['Cabin'].fillna('0', inplace = True)

In [249]:
data['Cabin'][1].values[0][:1]

'C'

In [250]:
data['CabinFL'] = data['Cabin'].map(lambda c:c[0][:1])

In [251]:
data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,CabinFL
0,22.0,0,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr,0
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Mrs,C
2,26.0,0,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Miss,0
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Mrs,C
4,35.0,0,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Mr,0


In [252]:
pd.crosstab(data.Survived,data.CabinFL)

CabinFL,0,A,B,C,D,E,F,G,T
Survived,,,,,,,,,
0.0,481,8,12,24,8,8,5,2,1
1.0,206,7,35,35,25,24,8,2,0


##### Вероятно номер кабины был известен по большей части у тех, кто выжил

### Fare

In [253]:
test[test['Pclass']==3].info() # Пропущенное значение в 3 классе. ЗАменим его медианным значением по классу.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    218 non-null int64
Pclass         218 non-null int64
Name           218 non-null object
Sex            218 non-null object
Age            146 non-null float64
SibSp          218 non-null int64
Parch          218 non-null int64
Ticket         218 non-null object
Fare           217 non-null float64
Cabin          4 non-null object
Embarked       218 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 20.4+ KB


In [254]:
data.loc[data['Pclass']==3,'Fare'] = data.loc[data['Pclass']==3]['Fare'].fillna(data.loc[data['Pclass']==3].Fare.median())

In [255]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 14 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
CabinFL        1309 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 193.4+ KB


### Age

In [256]:
data["Age"] = data.groupby(['Sex','Pclass','Title'])['Age'].transform(lambda x: x.fillna(x.median()))

### Удалим ненужную информацию

In [257]:
data.drop(data.columns[[1,4,8,11,13]], axis=1, inplace= True)

In [258]:
data.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Title
0,22.0,S,7.2500,0,1,3,1,0.0,Mr
1,38.0,C,71.2833,0,2,1,1,1.0,Mrs
2,26.0,S,7.9250,0,3,3,0,1.0,Miss
3,35.0,S,53.1000,0,4,1,1,1.0,Mrs
4,35.0,S,8.0500,0,5,3,0,0.0,Mr


In [259]:
def data_to_numeric(df):
    Title_Dictionary = {
                        "Officer": 0,
                        "Royalty": 1,
                        "Mrs":     2,
                        "Miss":    3,
                        "Mr":      4,
                        "Master":  5
                       }
    
    Embarked_Dictionary = {
                            "S": 0,
                            "C": 1,
                            "Q": 2
                           }
        
    # we map each title and Embarked
    df['Title'] = df.Title.map(Title_Dictionary)
    df['Embarked'] = df.Embarked.map(Embarked_Dictionary)
    return df

In [260]:
def split(df):
    X = df[df.columns]    
    y = df['Survived']
    
    X.drop(['Survived'], axis=1, inplace=True)
    
    X_train = X[:891]
    X_test = X[891:] 
    y_train = y[:891]
    y_test = y[891:]
    X_train.head()
    return X_train, X_test, y_train, y_test

In [261]:
def submit(pred,model):
    submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": preds_class.astype('int64')
        })
    submission.to_csv('titanic_'+str(model)+'.csv', index=False)

## Logistic Regression

In [262]:
dataLR = data.copy()
data_to_numeric(dataLR)

,Age,Embarked,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Title
0,22.0,0,7.2500,0,1,3,1,0.0,4
1,38.0,1,71.2833,0,2,1,1,1.0,2
2,26.0,0,7.9250,0,3,3,0,1.0,3
3,35.0,0,53.1000,0,4,1,1,1.0,2
4,35.0,0,8.0500,0,5,3,0,0.0,4
5,29.0,2,8.4583,0,6,3,0,0.0,4
6,54.0,0,51.8625,0,7,1,0,0.0,4
7,2.0,0,21.0750,1,8,3,3,0.0,5
8,27.0,0,11.1333,2,9,3,0,1.0,2
9,14.0,1,30.0708,0,10,2,1,1.0,2


In [263]:
data.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Title
0,22.0,S,7.2500,0,1,3,1,0.0,Mr
1,38.0,C,71.2833,0,2,1,1,1.0,Mrs
2,26.0,S,7.9250,0,3,3,0,1.0,Miss
3,35.0,S,53.1000,0,4,1,1,1.0,Mrs
4,35.0,S,8.0500,0,5,3,0,0.0,Mr


In [264]:
dataLR.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 9 columns):
Age            1309 non-null float64
Embarked       1309 non-null int64
Fare           1309 non-null float64
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
SibSp          1309 non-null int64
Survived       891 non-null float64
Title          1309 non-null int64
dtypes: float64(3), int64(6)
memory usage: 102.3 KB


In [265]:
X_train, X_test, y_train, y_test = split(dataLR)

In [266]:
X_train.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,SibSp,Title
0,22.0,0,7.2500,0,1,3,1,4
1,38.0,1,71.2833,0,2,1,1,2
2,26.0,0,7.9250,0,3,3,0,3
3,35.0,0,53.1000,0,4,1,1,2
4,35.0,0,8.0500,0,5,3,0,4


In [267]:
# specify the training parameters 
model = LogisticRegression()
#train the model
model.fit(X_train, y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", preds_class[:10])
print("proba = ", preds_proba[:10])

class =  [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
proba =  [[ 0.76842956  0.23157044]
 [ 0.77344276  0.22655724]
 [ 0.85263836  0.14736164]
 [ 0.82477755  0.17522245]
 [ 0.48556471  0.51443529]
 [ 0.71889423  0.28110577]
 [ 0.5597834   0.4402166 ]
 [ 0.69098913  0.30901087]
 [ 0.32856217  0.67143783]
 [ 0.83728005  0.16271995]]


In [268]:
submit(preds_class,'LogReg')

## CatBoost

In [269]:
data.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Title
0,22.0,S,7.2500,0,1,3,1,0.0,Mr
1,38.0,C,71.2833,0,2,1,1,1.0,Mrs
2,26.0,S,7.9250,0,3,3,0,1.0,Miss
3,35.0,S,53.1000,0,4,1,1,1.0,Mrs
4,35.0,S,8.0500,0,5,3,0,0.0,Mr


In [270]:
dataCatBoost = data.copy()

In [271]:
dataCatBoost.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,SibSp,Survived,Title
0,22.0,S,7.2500,0,1,3,1,0.0,Mr
1,38.0,C,71.2833,0,2,1,1,1.0,Mrs
2,26.0,S,7.9250,0,3,3,0,1.0,Miss
3,35.0,S,53.1000,0,4,1,1,1.0,Mrs
4,35.0,S,8.0500,0,5,3,0,0.0,Mr


In [272]:
dataCatBoost.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Royalty', 'Officer'], dtype=object)

In [273]:
dataCatBoost.columns

Index(['Age', 'Embarked', 'Fare', 'Parch', 'PassengerId', 'Pclass', 'SibSp',
       'Survived', 'Title'],
      dtype='object')

In [274]:
CatBoost_X = dataCatBoost[['Age', 'Embarked', 'Fare', 'Parch', 'PassengerId', 'Pclass', 'SibSp', 'Title']]
CatBoost_Y = dataCatBoost['Survived']

In [275]:
CatBoost_X.shape

(1309, 8)

In [276]:
CatBoost_Y.shape

(1309,)

In [277]:
X_train, X_test, y_train, y_test = split(dataCatBoost)

In [278]:
X_train.columns

Index(['Age', 'Embarked', 'Fare', 'Parch', 'PassengerId', 'Pclass', 'SibSp',
       'Title'],
      dtype='object')

In [279]:
# specify the training parameters 
model = CatBoostClassifier(iterations=2, depth=2, learning_rate=1, loss_function='Logloss')
#train the model
model.fit(X_train, y_train, cat_features=[1,3,4,5,6,7])
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", preds_class[:10])
print("proba = ", preds_proba[:10])

class =  [ 0.  1.  0.  0.  1.  0.  1.  0.  1.  0.]
proba =  [[ 0.90395408  0.09604592]
 [ 0.39588923  0.60411077]
 [ 0.78732868  0.21267132]
 [ 0.90395408  0.09604592]
 [ 0.39588923  0.60411077]
 [ 0.90395408  0.09604592]
 [ 0.39588923  0.60411077]
 [ 0.78732868  0.21267132]
 [ 0.39588923  0.60411077]
 [ 0.90395408  0.09604592]]


In [280]:
preds_proba.shape

(418, 2)

In [281]:
test.shape

(418, 11)

In [282]:
submit(preds_class,'CatBoost')

### CatBoost_alldata

In [217]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
data_all = pd.concat([train, test])

In [218]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [219]:
get_titles(data_all)

In [220]:
data_all['Cabin'].fillna('U',inplace=True)

In [221]:
data_all['Embarked'].fillna('S', inplace = True)

In [222]:
data_all.loc[data_all['Pclass']==3,'Fare'] = data_all.loc[data_all['Pclass']==3]['Fare'].fillna(data_all.loc[data_all['Pclass']==3].Fare.median())

In [223]:
data_all['Age'] = data_all.groupby(['Sex','Pclass','Title'])['Age'].transform(lambda x: x.fillna(x.median()))
X_train, X_test, y_train, y_test = split(data_all)

In [224]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 143.2+ KB


In [225]:
X_train.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Ticket,Title
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,A/5 21171,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,PC 17599,Mrs
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,STON/O2. 3101282,Miss
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,113803,Mrs
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,373450,Mr


In [226]:
# CatBoost_alldata
# specify the training parameters 
model = CatBoostClassifier(iterations=2, depth=2, learning_rate=1, loss_function='Logloss')
#train the model
model.fit(X_train, y_train, cat_features=[1,2,4,5,6,7,8,9,10,11])
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", preds_class[:10])
print("proba = ", preds_proba[:10])

class =  [ 0.  1.  0.  0.  1.  0.  1.  0.  1.  0.]
proba =  [[ 0.88225463  0.11774537]
 [ 0.488754    0.511246  ]
 [ 0.79110098  0.20889902]
 [ 0.88225463  0.11774537]
 [ 0.488754    0.511246  ]
 [ 0.88225463  0.11774537]
 [ 0.488754    0.511246  ]
 [ 0.79110098  0.20889902]
 [ 0.488754    0.511246  ]
 [ 0.88225463  0.11774537]]


In [227]:
submit(preds_class,'CatBoost_alldata')

## XGBoost

In [63]:
import xgboost

## DecisionTree

## RandomForest